In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""
사용 가능한 장치 얻기.
"""

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
"""
신경망 만들기.

nn.Module을 상속받아야 하며 순전파forward 메서드를 구현해야 한다.
nn.Linear, nn.ReLU 모두 nn.Module을 상속받은 클래스이다.
"""

class NeuralNetwork(nn.Module):

	def __init__(self):
		super().__init__()
		self.flatten = nn.Flatten()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(28*28, 512),
			nn.ReLU(),
			nn.Linear(512, 512),
			nn.ReLU(),
			nn.Linear(512, 10),
		)
	
	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)
		return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [4]:
"""
신경망의 Parameter 살펴보기.
"""

for name, param in model.named_parameters():
	print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0240, -0.0214,  0.0036,  ...,  0.0232, -0.0303,  0.0170],
        [ 0.0099, -0.0113, -0.0112,  ...,  0.0277,  0.0347, -0.0214]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0121, -0.0141], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0146, -0.0003,  0.0235,  ...,  0.0327,  0.0051, -0.0439],
        [-0.0239, -0.0432,  0.0165,  ..., -0.0130, -0.0411, -0.0117]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([-0.0286, -0.0338], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0437, -0.0114, -0.0183,  ...,  0.0117, -0.0154,  0.0029],
        [-0.0411, -0.0094,  0.0413,

In [5]:
"""
텐서, nn.Module을 만들고 살펴보기.
"""

X = torch.randn(3, 28, 28, device=device)
print("X.shape:", X.shape, type(X), X.device)
print("model.flatten(X).shape:", model.flatten(X).shape)

layer1 = nn.Linear(in_features=28*28, out_features=20).to(device)
print("layer1:", layer1, type(layer1))

hidden1 = layer1(model.flatten(X))
print("hidden1.shape:", hidden1.shape, type(hidden1), hidden1.device)

X.shape: torch.Size([3, 28, 28]) <class 'torch.Tensor'> cuda:0
model.flatten(X).shape: torch.Size([3, 784])
layer1: Linear(in_features=784, out_features=20, bias=True) <class 'torch.nn.modules.linear.Linear'>
hidden1.shape: torch.Size([3, 20]) <class 'torch.Tensor'> cuda:0


In [6]:
"""
ReLU 비선형 활성화 함수 사용해보기.
"""

x = torch.randn(5)
relu = nn.ReLU()
print("Before ReLU:", x)
x = relu(x)
print("After ReLU:", x)

Before ReLU: tensor([ 1.4497, -0.6941,  0.6914, -0.3930, -0.2554])
After ReLU: tensor([1.4497, 0.0000, 0.6914, 0.0000, 0.0000])


In [7]:
"""
Softmax 사용해보기.
"""

X = torch.randn(3, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3, 3, 3], device='cuda:0')
